In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
#json数据

In [4]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [5]:
import json

In [6]:
result = json.loads(obj)    #使用json.loads方法将JSON字符串转换为Python形式

In [7]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [8]:
asjson = json.dumps(result)    #将Python对象转为JSON

In [9]:
siblings = pd.DataFrame(result['siblings'], columns = ['name', 'age'])#将字典构造的列表(之前是JSON对象)传入构造函数，并选出数据字段的子集

In [10]:
siblings

,name,age
0,Scott,30
1,Katie,38


In [11]:
!type examples\example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [12]:
data = pd.read_json('examples/example.json')    #pandas.read_json可以自动将JSON数据集按照指定次序转换为Series或DataFrame

In [13]:
data       #pandas.read_json的默认选项是假设JSON数组中的每个对象是表里的一行

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [14]:
print(data.to_json())            #可以对Series和DataFrame使用to_json方法从pandas中将数据导出为JSON

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [15]:
print(data.to_json(orient = 'records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


In [16]:
#XML和HTML:网络抓取

In [17]:
#pandas的内建函数read_html可以使用lxml和Beautiful Soup等库将HTML中的表自动解析为DataFrame对象

In [18]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [19]:
len(tables)

1

In [20]:
failures = tables[0]

In [21]:
failures.head()       

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [22]:
close_timestamps = pd.to_datetime(failures['Closing Date']) 

In [23]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

In [24]:
#使用lxml.objectify解析XML

In [25]:
from lxml import objectify

In [26]:
path = 'examples/mta_perf/Performance_MNR.xml'

In [27]:
parsed = objectify.parse(open(path))      #解析文件

In [28]:
root = parsed.getroot()           #getroot获得对XML文件根节点引用

In [29]:
data = []

In [30]:
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

In [31]:
#root.INDICATOR返回一个生成器，可以产生每一个<INDICATOR>XML元素。对于每条记录，可以将标签名称的字典(如YTD_ACTUAL)填充为数据值(不包括几个标签)

In [32]:
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [33]:
perf = pd.DataFrame(data)

In [34]:
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [35]:
from io import StringIO

In [36]:
tag = '<a href = "http:// www.google.com">Google</a>'

In [37]:
root = objectify.parse(StringIO(tag)).getroot()

In [38]:
root

<Element a at 0x1f3b9af3d08>

In [39]:
root.get('href')

'http:// www.google.com'

In [40]:
root.text

'Google'

In [41]:
#二进制格式

In [42]:
#pickle仅被推荐作为短期的存储格式，高效方便

In [43]:
frame = pd.read_csv('examples/ex1.csv')    

In [44]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [45]:
frame.to_pickle('examples/frame_pickle')  #to_pickle方法可以将数据以pickle格式写入硬盘(序列化)

In [46]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [47]:
#使用HDF5格式   

In [48]:
#HDF代表分层数据格式，每个HDF5文件可以存储多个数据集并支持元数据，有其他语言接口包括Java, Julia, MATLABA, Python

In [49]:
#适合处理不适合在内存中存储的超大型数据，可以高效的读写大型数组的一小块

In [50]:
frame = pd.DataFrame({'a' : np.random.randn(100)})

In [51]:
store = pd.HDFStore('mydata.h5')      #HDFStore类像字典一样工作并处理低级别细节

In [52]:
store['obj1'] = frame

In [53]:
store['obj1_col'] = frame['a']

In [54]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [55]:
pd.options.display.max_rows = 10

In [56]:
store['obj1']      #包含在HDF5文件中的对象可以使用相同的字典型API进行检索

,a
0,0.349510
1,0.936218
2,0.673225
3,0.544063
4,0.097749
...,...
95,-0.084757
96,1.393052
97,-1.454476
98,-1.190374


In [57]:
#HDFStore支持两种存储模式，'fixed' 和'table'，后者速度慢，单支持一种特殊语法的查询操作：

In [58]:
store.put('obj2', frame, format = 'table')

In [59]:
store.select('obj2', where = ['index >= 10 and index <= 15'])

,a
10,0.514666
11,0.605507
12,-0.359403
13,-0.299397
14,-1.016759
15,0.291809


In [60]:
store.close()

In [61]:
#put是store['obj2']=frame方法的显式版本，可以设置其他选项，如存储格式

In [62]:
frame.to_hdf('mydata.h5', 'obj3', format = 'table')

In [63]:
pd.read_hdf('mydata.h5', 'obj3', where = ['index < 5'])

,a
0,0.349510
1,0.936218
2,0.673225
3,0.544063
4,0.097749


In [64]:
#读取microsoftExcel文件

In [65]:
#pandas支持通过ExcelFile类或pandas.read_excel函数读取存在Excel中的表格型数据

In [67]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')    #生成实例

In [68]:
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [69]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

In [70]:
frame               #读取含有多个表的文件是，生成ExcelFile更快

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [85]:
writer = pd.ExcelWriter('examples/ex22.xlsx')       #生成一个ExcelWriter对象

In [86]:
frame.to_excel(writer, 'Sheet1')                 #写入数据

In [87]:
writer.save()

In [76]:
frame.to_excel('examples/ex2.xlsx')

In [88]:
#与Web API交互

In [90]:
import requests

In [91]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [92]:
resp = requests.get(url)

In [93]:
resp

<Response [200]>

In [94]:
data = resp.json()         

In [95]:
data[0]['title']

'Series.update fails with categorical types'

In [100]:
issues = pd.DataFrame(data, columns = ['number', 'title', 'labels', 'state'])

In [101]:
issues

,number,title,labels,state
0,25744,Series.update fails with categorical types,[],open
1,25743,BUG: Fix groupby with MultiIndex Series corner...,[],open
2,25738,Dropna argument is now respected when false in...,"[{'id': 2822342, 'node_id': 'MDU6TGFiZWwyODIyM...",open
3,25737,Resample with freq less than 10T generates dif...,[],open
4,25736,Fix incorrect example in wide_to_long docstring,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
...,...,...,...,...
25,25693,Replace dicts with OrderedDicts in groupby agg...,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
26,25692,Using OrderedDict in GroupBy Module(s),"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
27,25691,BUG: Segmentation fault using tuple as iterato...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
28,25690,TST/CLN: empty DataFrames and some 'empty' Series,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open


In [102]:
#与数据库交互

In [109]:
import sqlalchemy as sqla      #Python SQL工具包

In [112]:
#数据库+数据库连接框架://用户名:密码@IP地址:端口号/数据库名称
db = sqla.create_engine('mysql+pymysql://root:@localhost:3306/spider')

In [114]:
pd.read_sql('select * from 360images', db)

,id,url,title,thumb
0,02d807137db09e200e474b65cf9e0404,https://ps.ssl.qhmsg.com/t01109fad525ea8ecff.jpg,?????,https://ps.ssl.qhmsg.com/sdr/238__/t01109fad52...
1,04039f0cd302e972098a45c45cc5eb19,https://ps.ssl.qhmsg.com/t012de97454cd99094f.jpg,????·???????????????????,https://ps.ssl.qhmsg.com/sdr/238__/t012de97454...
2,060a2343e9c70f94d7cc3893980b4c51,https://ps.ssl.qhmsg.com/t019bec4cb57d06f015.jpg,??,https://ps.ssl.qhmsg.com/sdr/238__/t019bec4cb5...
3,06a575dd981336a9bbc6021f7fbbfb9a,https://p3.ssl.qhimgs1.com/t0162dbeebfa728c763...,09-7-16?7-26??&amp;???2,https://p3.ssl.qhimgs1.com/sdr/238__/t0162dbee...
4,06b90da346f1525bd153ee4665b42254,https://ps.ssl.qhmsg.com/t0148ffdc5f517a736f.jpg,?????2?????1????????------???,https://ps.ssl.qhmsg.com/sdr/238__/t0148ffdc5f...
...,...,...,...,...
145,f63c460686a842b978c0cd7e2ed2f147,https://p3.ssl.qhimgs1.com/t0187a67d31855ad163...,2010? ?????,https://p3.ssl.qhimgs1.com/sdr/238__/t0187a67d...
146,f755c48c1a94c9c49d11aac94c69184f,https://ps.ssl.qhmsg.com/t01ea0021592a32a55f.jpg,???????????????,https://ps.ssl.qhmsg.com/sdr/238__/t01ea002159...
147,f813ca446cf424b148e8fbb19f8c99fc,https://p5.ssl.qhimgs1.com/t01ae0f36e8784129eb...,???????,https://p5.ssl.qhimgs1.com/sdr/238__/t01ae0f36...
148,f870ebf78f98446d4afebf23e24fb9ae,https://ps.ssl.qhmsg.com/t010d91cbdac35a775f.jpg,??????,https://ps.ssl.qhmsg.com/sdr/238__/t010d91cbda...
